# Exercise: Creating the Bob In-Flight HTLC Commitment Transaction

Now as exercise let's create the Bob In-Flight Commitment Transaction.

In [1]:
%run "../chapter 2 - initial commitment transaction/initial commitment transaction.ipynb"

2026-02-13T19:28:34.428000Z TestFramework (INFO): PRNG seed is: 4137849840964469644
2026-02-13T19:28:34.429000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_jtnmr6xi
🟢 New TestShell started. Block height: 0
Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Payment BasePoint:  025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Bob Payment BasePoint:  02f98efd3f2b2fbe7bd83c419f5f64f8280798b8a9175fdb77c0091bbb95c79506
To obscure commitment number 0xb433fd43a66f
Alice funding pubkey: 649c22aa0a8357736a04ea9b8ac3ba4fad84052ba661473c6d924b22125abe38
Alice funding privkey: e8abdff4b46a5486832f6b022e0a32278f69138ebb35e535add0faf56b7c43e7
Alice funding address: bcrt1pvjwz92s2sdthx6sya2dc4sa6f7kcgpft5es5w0rdjf9jyyj6hcuqu7j998
Alice sweeper pubkey: 3e2129e9941879d714d4af438afcf4feaedf1765547b05a8256a3e797687a24e
Alice s

## The Unsigned Transaction

### The Input

In [2]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(funding_channel_txid))[::-1]
funding_channel_index = 0
index = funding_channel_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are the upper 24 bits of the obscured commitment number
# Commitment number of the second commitment transaction.
commitment_number = 1
# obscured commitment number is result of xor operation 
commitment_number_obscured = lower48_to_obscure ^ commitment_number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
# Convert to little-endian 
sequence = sequence[::-1]

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

print("Commitment Number Osbscured", hex(commitment_number_obscured))

Commitment Number Osbscured 0xb433fd43a66e


### The Outputs

Bob transaction will have 4 outputs also:
* local anchor output
* remote anchor output
* accepted htlc output
* to remote output

#### The Accepted HTLC Output

    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +-------------------+
                    ---| P(revocation) + T |
                       +-------------------+
                                         ^
                                         |
                                   +-----------+        
                                   | T = t * G |
                                   +-----------+        
                                         ^
                                         |
     +---+   +-------------------------------------------------------+
     | t | = | TaggedHash ("Taptweak", P(revocation) || script_root) |
     +---+   +-------------------------------------------------------+
                                                             ^
                                                             |
                                                          +-----+
                          ------------------------------> | hAB |<------------------
                         |                                +-----+                   |
                         |                                                          |
                         |                                                          |
                      +----+                                                     +----+
                      | hA |                                                     | hB |
                      +----+                                                     +----+
                         ^                                                          ^
                         |                                                          |
      +----------------------------------------------------+                        |
      | P(remote_htlc) OP_CHECKSIG OP_1 OP_CSV OP_DROP     |                        |
      |                                                    |                        |
      | cltv_expiry OP_CHECKLOCKTIMEVERIFY OP_DROP         |                        |
      +----------------------------------------------------+                        |
      +-------------------------------------------------------------------------------+ 
      | OP_SIZE 32 OP_EQUALVERIFY OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY |
      |                                                                               |
      | P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG                    |
      +-------------------------------------------------------------------------------+


In [3]:
# Create Alice per-commitment
alice_per_commitment = per_commitment(alice_per_commitment_seed, commitment_number)
# Create Bob per-commitment
bob_per_commitment = per_commitment(bob_per_commitment_seed, commitment_number)

# Create Alice Local Public Key which in this case is the P(remote) for Bob
# Use the method 'get_pubkey' from the basepoint opject (alice payment) and the alice per commitment point to derive the alice payment pubkey
alice_payment_pubkey = alice_payment.get_pubkey(alice_per_commitment.get_pub())
print(f"Alice Payment PubKey: {alice_payment_pubkey.get_bytes(bip340=True).hex()}")

# Create Bob Delayed Public Key
bob_delayed_pubkey = derivate_key(bob_node_seed, family=4, channel_index=0).get_pubkey(bob_per_commitment.get_pub())
print(f"Bob Delayed PubKey: {bob_delayed_pubkey.get_bytes(bip340=True).hex()}")

# Create Alice Revocation Public Key
alice_revocation_pubkey = derivate_revocation_key(alice_node_seed, channel_index=0).get_pubkey(alice_per_commitment.get_pub())
print(f"Alice Revocation PubKey: {alice_revocation_pubkey.get_bytes(bip340=True).hex()}")

# Create Alice HTLC Public Key
alice_htlc_pubkey = derivate_key(alice_node_seed, family=2, channel_index=0).get_pubkey(alice_per_commitment.get_pub())
print(f"Alice HTLC PubKey: {alice_htlc_pubkey.get_bytes(bip340=True).hex()}")

# Create Bob HTLC Public Key
bob_htlc_pubkey = derivate_key(bob_node_seed, family=2, channel_index=0).get_pubkey(bob_per_commitment.get_pub())
print(f"Bob HTLC PubKey: {bob_htlc_pubkey.get_bytes(bip340=True).hex()}")

# Outputs for Bob In-Flight Commitment Transaction
# 0x04 outputs
output_count = bytes.fromhex("04")

# ANCHOR OUTPUTS
# Anchor output amount (we use the same amount as before)

# OP_16 OP_CSV
script = CScript([OP_16, OP_CHECKSEQUENCEVERIFY])

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# Anchor Output script_root
script_root = tagged_hash("TapBranch", hash_input)

# Alice Anchor Output Tagged Hash (remote anchor for Bob)
taptweak = tagged_hash("TapTweak", alice_payment_pubkey.get_bytes(bip340=True) + script_root)
alice_payment_pubkey_tweaked = alice_payment_pubkey.tweak_add(taptweak)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_anchor_spk = bytes.fromhex("51") + varint_len(alice_payment_pubkey_tweaked.get_bytes(bip340=True)) + alice_payment_pubkey_tweaked.get_bytes(bip340=True)

# Bob Anchor Output Tagged Hash (local anchor for Bob)
taptweak = tagged_hash("TapTweak", bob_delayed_pubkey.get_bytes(bip340=True) + script_root)
bob_delayed_pubkey_tweaked = bob_delayed_pubkey.tweak_add(taptweak)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
bob_anchor_spk = bytes.fromhex("51") + varint_len(bob_delayed_pubkey_tweaked.get_bytes(bip340=True)) + bob_delayed_pubkey_tweaked.get_bytes(bip340=True)

# Sort by scriptPubKey (lexicographic order)
anchors = [
    (alice_anchor_spk, "alice_anchor"),
    (bob_anchor_spk,   "bob_anchor"),
]

anchors_sorted = sorted(anchors, key=lambda x: x[0])

# ACCEPTED HTLC OUTPUT
# HTLC output amount
htlc_output_value_satoshis = 500000
htlc_output_value = htlc_output_value_satoshis.to_bytes(8, byteorder="little", signed=False)

# Create the leaf scripts A and B
# script A: P(remote_htlc) OP_CHECKSIG OP_1 OP_CSV OP_DROP cltv_expiry OP_CHECKLOCKTIMEVERIFY OP_DROP
# Add cltv_delta (40 blocks) to current block height
cltv_expiry = node.getblockcount() + 40
scriptA = CScript([alice_htlc_pubkey.get_bytes(bip340=True), OP_CHECKSIG, OP_1, OP_CHECKSEQUENCEVERIFY, cltv_expiry, OP_CHECKLOCKTIMEVERIFY, OP_DROP])

# We use a dummy payment_hash here. In a real scenario, this would be the hash of the preimage for the HTLC.
# And would be sent in the message update_add_htlc.
payment_preimage = sha256(b'payment_secret').digest()
sha = sha256(payment_preimage).digest()
payment_hash = hashlib.new("ripemd160", sha).digest()

# Leaf script B: OP_SIZE 32 OP_EQUALVERIFY OP_HASH160 <RIPEMD160(payment_hash)> OP_EQUALVERIFY P(local_htlc) OP_CHECKSIGVERIFY P(remote_htlc) OP_CHECKSIG
# We use the same dummy payment hash as before.
scriptB = CScript([OP_SIZE, bytes.fromhex("20"), OP_EQUALVERIFY, OP_HASH160, payment_hash, OP_EQUALVERIFY, bob_htlc_pubkey.get_bytes(bip340=True), OP_CHECKSIGVERIFY, alice_htlc_pubkey.get_bytes(bip340=True), OP_CHECKSIG])

# Compute TapLeaves A and B
# Method: ser_string(data) is a function which adds compactsize to input data.
hash_inputA = TAPSCRIPT_VER + ser_string(scriptA)
hash_inputB = TAPSCRIPT_VER + ser_string(scriptB)

# Use the tagged_hash function to compute TapLeaf A and B
htlc_taggedhash_leafA = tagged_hash("TapLeaf", hash_inputA)
taggedhash_leafB = tagged_hash("TapLeaf", hash_inputB)

# Method: Returns tapbranch hash. Child hashes are lexographically sorted and then concatenated.
# l: tagged hash of left child
# r: tagged hash of right child
def tapbranch_hash(l, r):
    return tagged_hash("TapBranch", b''.join(sorted([l,r])))

# Compute Internal node TapBranch AB
root_ab = tapbranch_hash(htlc_taggedhash_leafA, taggedhash_leafB)

# Compute TapTweak
taptweak = tagged_hash("TapTweak", alice_revocation_pubkey.get_bytes(bip340=True) + root_ab)

# Tweak the alice revocation pubkey
alice_revocation_pubkey_tweaked = alice_revocation_pubkey.tweak_add(taptweak)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
bob_accepted_htlc_spk = bytes.fromhex("51") + varint_len(alice_revocation_pubkey_tweaked.get_bytes(bip340=True)) + alice_revocation_pubkey_tweaked.get_bytes(bip340=True)

# Second commitment expected weight 
commitment_weight = taproot_commit_weight(NumTapOut=2, NumAnchorOut=2)
print("Commitment Transaction Expected Weight:", commitment_weight)

# From open_channel and update_feerate messages (1 sat/vb = 250 sat/kw)
feerate_per_kw = 250

# Calculate the fee for the second commitment transaction
commitment_fee = int(commitment_weight * feerate_per_kw / 1000)

# TO REMOTE OUTPUT
# to remote output amount
# As we have no to_local output in this case, the to_remote value pay the fees
# Using the same fee as before
to_remote_value_sat =  channel_value_sat - anchor_output_value_satoshis * 2 - htlc_output_value_satoshis - commitment_fee
to_remote_value = to_remote_value_sat.to_bytes(8, byteorder="little", signed=False)
print("channel_value_sat, anchor_output_value_satoshis * 2, commitment_fee: ", channel_value_sat, anchor_output_value_satoshis, commitment_fee)

# P(remote) OP_CHECKSIG OP_1 OP_CSV OP_DROP
# Use CScript to create the to remote script
script = CScript([alice_payment_pubkey.get_bytes(bip340=True), OP_CHECKSIG, OP_1, OP_CHECKSEQUENCEVERIFY, OP_DROP])

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# To Remote Output script_root
# Use the tagged_hash function to compute the script root
script_root = tagged_hash("TapLeaf", hash_input)

# To Remote Output Tagged Hash
# Use the tagged_hash function to compute the taptweak
taptweak = tagged_hash("TapTweak", NUMS.get_bytes(bip340=True) + script_root)
# Use the tweak_add to tweak the NUMS key
NUMS_tweaked = NUMS.tweak_add(taptweak)
# Create the scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
bob_to_remote_spk = bytes.fromhex("51") + varint_len(NUMS_tweaked.get_bytes(bip340=True)) + NUMS_tweaked.get_bytes(bip340=True)

outputs = (
    anchor_output_value
    + varint_len(anchors_sorted[0][0])
    + anchors_sorted[0][0]
    + anchor_output_value
    + varint_len(anchors_sorted[1][0])
    + anchors_sorted[1][0]
    + htlc_output_value
    + varint_len(bob_accepted_htlc_spk)
    + bob_accepted_htlc_spk
    + to_remote_value
    + varint_len(bob_to_remote_spk)
    + bob_to_remote_spk
)

# Locktime is the same

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))


Alice Payment PubKey: 4f117d26e658857062d1b3f3e2af220fedd2c524e100f4e1afa43226a5b3ac70
Bob Delayed PubKey: ad6870c3a077f952a43320ab2750f8b919f7a6fc021044c3cb8f8cef72f498dc
Alice Revocation PubKey: 02c3bcb049c8059227f85f26805179b4ff6e0573072696e4740f59cc0e21e37f
Alice HTLC PubKey: 7546c6ab3a8c005ce24e9feb7154fd35b62ac0f9125beb5e692d4d912a95de77
Bob HTLC PubKey: 7bd8383df06640c0efe760bb2bac1a1585bdc664e6f1cb61c41a76a6f2be3484
Commitment Transaction Expected Weight: 968
channel_value_sat, anchor_output_value_satoshis * 2, commitment_fee:  1000000 330 242
unsigned_tx:  0200000001ce08a991e8e57e540cb9f66270ba4c9374ddebc7a0284ef86952757d073e86920000000000fd33b480044a01000000000000225120f34b9e31cb06b69f932b8eb310861ffc1b125114b6eb2c00fe050bb37197f2be4a01000000000000225120f6b0e14f57145394ebdfd6bb45da27de360930b64cc65bdec422e8fec286103720a10700000000002251204cd5b607b8c1680cbd82425ff0029716c3ce0269155e03b8b5d3af14a4c722bb9a9d070000000000225120245114f720781ddf5b1d5eaa987c2af12e9eee22510c7dcb8255ef

## The sighash for the key path spend

In [4]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index).digest()

sha_amounts = sha256(channel_value).digest()

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
sha_scriptpubkeys = sha256(
    varint_len(channel_spk)
    + channel_spk
).digest()

sha_sequences = sha256(sequence).digest()

sha_outputs = sha256(outputs).digest()

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()


## Signing the sighash

In [5]:
# Reusing the reordered participants to match the sorted pubkeys

# Alice and Bob generates its own nonce pair (secnonce, pubnonce)
secnonces, pubnonces = [], []
for pk, sk, pcs in participants:
    # Use per-commitment nonce for bob to get deterministic results and nonce_gen for Alice jit nonce
    if pk == pubkey_bob_musig2.get_bytes(bip340=False):
        sec, pub = nonce_per_commitment(pcs, commitment_number, sk, pk, agg_pubkey_tweaked, sighash)
    else:
        sec, pub = nonce_gen(sk, pk, agg_pubkey_tweaked, sighash, None)
    secnonces.append(sec)
    pubnonces.append(pub)

# Alice and Bob has exchanged already the pubnonces, so they can agregate them and create the session context
# Use nonce_agg to aggregate the pubnonces
agg_nonce = nonce_agg(pubnonces)
# Use the SessionContext to create the gignature session
session = SessionContext(agg_nonce, sorted_pubkeys, [bip86_tweak], [True], sighash)

# Alice and Bob produces their partial signatures
# Use sign function to produce the partial signatures
psigs = [sign(sec, sk, session) for sec, (_, sk, _) in zip(secnonces, participants)]

# Each side verify the other’s partial signature before proceeding
for i, psig in enumerate(psigs):
    assert partial_sig_verify(psig, pubnonces, sorted_pubkeys, [bip86_tweak], [True], sighash, i)

# Each side combines partial signatures into the final Schnorr signature
# Use the function partial_sig_agg to aggregate the partial signatures
agg_sig = partial_sig_agg(psigs, session)

# Sanity check: verify with BIP340 against the *tweaked* x-only key ---
ok = schnorr_verify(sighash, agg_pubkey_tweaked, agg_sig)
print("aggregated Schnorr verifies?", ok)


aggregated Schnorr verifies? True


## The signed transaction

In [6]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(agg_sig)
    + agg_sig
)

# the final signed transaction
signed_bob_commitment_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_bob_commitment_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_bob_commitment_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

print(node.testmempoolaccept(rawtxs=[signed_bob_commitment_tx.hex()]))

signed tx:  02000000000101ce08a991e8e57e540cb9f66270ba4c9374ddebc7a0284ef86952757d073e86920000000000fd33b480044a01000000000000225120f34b9e31cb06b69f932b8eb310861ffc1b125114b6eb2c00fe050bb37197f2be4a01000000000000225120f6b0e14f57145394ebdfd6bb45da27de360930b64cc65bdec422e8fec286103720a10700000000002251204cd5b607b8c1680cbd82425ff0029716c3ce0269155e03b8b5d3af14a4c722bb9a9d070000000000225120245114f720781ddf5b1d5eaa987c2af12e9eee22510c7dcb8255efc006680ac701400ddd76dfe0ad77d8ceac3388b00d6e7a398389815c93dd89ce681715fd85e43069de96dff0aca1ce0b8dee5955519966b390569030133307fd2b2536d49fa2586fa64320
{
  "txid": "1309ea834a64e5f0fac45bf03b0625142b5eda8303626566c5ee58d18e8a9ca4",
  "hash": "c3c6b01c27f300b6215a3db196eef3e3607abf5b1eb3ecca71c5c8e73c8b995b",
  "version": 2,
  "size": 291,
  "vsize": 240,
  "weight": 960,
  "locktime": 541304431,
  "vin": [
    {
      "txid": "92863e077d755269f84e28a0c7ebdd74934cba7062f6b90c547ee5e891a908ce",
      "vout": 0,
      "scriptSig": {
        "asm": "",
  